In [20]:
!pip install -q torch langchain bitsandbytes accelerate transformers sentence-transformers faiss-gpu >> None

In [21]:
!pip install langchain-community >> None

In [22]:
import os
import torch
import transformers

from glob import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,pipeline
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser

import warnings
warnings.simplefilter('ignore')

In [23]:
# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    print('Using GPU: ', torch.cuda.get_device_name(0))
    print('Memory Usage: ')
    print('Allocated: ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

else:
    print("GPU is not available.")

GPU is available.
Using GPU:  Tesla T4
Memory Usage: 
Allocated:  3.0 GB
Cached:  3.5 GB


### Setting bitsandbytes config to improve speed

In [24]:
!pip install pypdf >> None

In [25]:
!pip install peft -qU >> None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

### Loading a Quantized Mistral-7B Model

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate

adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"

tokenizer = AutoTokenizer.from_pretrained(
              base_model_name,
              trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
device_map = {"": 0}

model = AutoPeftModelForCausalLM.from_pretrained(
              adapt_model_name,
              device_map=device_map,
              torch_dtype=torch.bfloat16)

In [26]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/rugpt_large_turbo_instructed")
model = AutoModelForCausalLM.from_pretrained("IlyaGusev/rugpt_large_turbo_instructed")
model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/225 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50258, bias=False)
)

In [27]:
from langchain.llms import HuggingFacePipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature= 0.2,
    repetition_penalty=1.6,
    return_full_text=True,
    max_new_tokens=300,
    top_p= 0.2
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [28]:
prompt_template = """
"Ты — cпециалист службы поддержки.Ты разговариваешь с людьми и помогаешь им в решении их проблем.
Используй только следующий контекст, чтобы очень кратко ответить на вопрос в конце.
Не пытайся выдумывать ответ. Ответ запиши после строчки Answer:

Context:{context}

Question:{question}
Answer:
 """
# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [29]:
# Create llm chain
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
!unzip -q /content/НПА-20240615T052648Z-001.zip

replace НПА/Инструкция администратора Настройка ограничения прав доступа пользователей по ЦФО и статьям бюджетов для АО Гринатом.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [30]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

directory_path = "/content/НПА"

loader = DirectoryLoader(
    directory_path,
    glob="*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

100%|██████████| 30/30 [01:07<00:00,  2.26s/it]


In [31]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import clear_output

texts = [doc.page_content for doc in documents]

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", ".", "!", "?"],
    chunk_size=1977,
    chunk_overlap=61,
    length_function=len
)

split_documents = splitter.create_documents(texts)
print(len(split_documents))

embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = FAISS.from_documents(split_documents, embedding_model)
clear_output()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!unzip -q /content/Atomic-20240615T055822Z-001.zip

replace Atomic/NPA_db/index.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [37]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import clear_output

embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db= FAISS.load_local("/content/Atomic/NPA_db", embeddings, allow_dangerous_deserialization=True)

#query = "в УПП была кнопка для заполненияa колонки в ТЧ документа одним значением На скриншотах документы ``Передача материалов в кладовую``, ``Заказ материалов в производство`` возможна реализация в рамках официального ЗИ - нужно оценить трудозатраты --------------------- Добрый день! Имеется ввиду в документах по строкам «заполнение по реквизитам» , как было в УПП, например в заказе материалов не выбирать   в каждой позиции дату отгрузки.  X. В документах, например, «заказ поставщику», нет возможности как в XСERP заполнять строки аналитик «по реквизитам», просим добавить такую возможность в документы. Просьба пояснить, что имеется ввиду  -----"
query = "К кому можно обратиться с проблемами насчет документации"
query_vector = embeddings.embed_query(query)

search_results = db.similarity_search(query, k=30)

In [38]:
# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [39]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

### Testing model response with RAG

In [40]:
#query = "в УПП была кнопка для заполнения колонки в ТЧ документа одним значением На скриншотах документы ``Передача материалов в кладовую``, ``Заказ материалов в производство`` возможна реализация в рамках официального ЗИ - нужно оценить трудозатраты --------------------- Добрый день! Имеется ввиду в документах по строкам «заполнение по реквизитам» , как было в УПП, например в заказе материалов не выбирать   в каждой позиции дату отгрузки.  X. В документах, например, «заказ поставщику», нет возможности как в XСERP заполнять строки аналитик «по реквизитам», просим добавить такую возможность в документы. Просьба пояснить, что имеется ввиду  -----"
query = "К кому можно обратиться с проблемами насчет документации"
response = rag_chain.invoke(query)

In [41]:
print(response["text"])


"Ты — cпециалист службы поддержки.Ты разговариваешь с людьми и помогаешь им в решении их проблем.
Используй только следующий контекст, чтобы очень кратко ответить на вопрос в конце.
Не пытайся выдумывать ответ. Ответ запиши после строчки Answer:

Context:[Document(page_content='Имеется возможность  настройки отбора по периоду отсутствия \n'), Document(page_content='При необходимо-\nсти можно скорректировать сумму передачи. '), Document(page_content='В зависимости от того, какой именно \nсубсчет 76 -ого счета передается этой обработкой, в данный момент мог ут быть \n'), Document(page_content='В \nобработке предусмотрены отборы для возможности отбо ра тех заявок, \n')]

Question:К кому можно обратиться с проблемами насчет документации
Answer:
 Служба технической помощи пользователей может помочь вам решить любые проблемы связанные со службой поддержке клиентов или документацией вашей компании."igned", "quickly".="could help you solve any problems with your service support or documentati

#Optuna

In [ ]:
!pip install optuna rouge pymorphy2 -qU >> None

In [ ]:
!pip install --quiet langchain_experimental >> None

In [ ]:
!pip install sentence-transformers >> None

In [ ]:
!pip install faiss-cpu -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 57.4 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import clear_output

embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#faiss_index_qa = FAISS.load_local("MyDrive/Atomic/QA_db", embeddings, allow_dangerous_deserialization=True)
faiss_index_qa = FAISS.load_local("/content/Atomic/QA_db", embeddings, allow_dangerous_deserialization=True)


query = "в УПП была кнопка для заполненияa колонки в ТЧ документа одним значением На скриншотах документы ``Передача материалов в кладовую``, ``Заказ материалов в производство`` возможна реализация в рамках официального ЗИ - нужно оценить трудозатраты --------------------- Добрый день! Имеется ввиду в документах по строкам «заполнение по реквизитам» , как было в УПП, например в заказе материалов не выбирать   в каждой позиции дату отгрузки.  X. В документах, например, «заказ поставщику», нет возможности как в XСERP заполнять строки аналитик «по реквизитам», просим добавить такую возможность в документы. Просьба пояснить, что имеется ввиду  -----"
query_vector = embeddings.embed_query(query)

search_results = faiss_index_qa.similarity_search(query, k=3)

In [ ]:
search_results

[Document(page_content='              Добрый день! на основе заказа клиента был создан заказ на производство. подразделение и в заказе клиента и в заказе на производство заполнено. так почему в поле ``спецификация`` продтягивается совершенно другое подразделение? (скрины во вложении)                            зачем мы в самой спецификации указываем подразделение-диспетчер, если это всё равно не играет никакой роли? к чему это автозаполнение, если оно некорректное?'),
 Document(page_content='В планах закупки был создан заказ на товар, вид плана- заявка SRM. Во вкладке Товары, есть поле для заполнения - Назначение, в которой выпадает строка - Подразделение ОАХО и дата (заказ на внутр.потребление), дата за прошлый год. Непонятно что за информацию я должна вносить, данное поле я должна заполнять из выпадающего списка?'),
 Document(page_content='Шаги воспроизведения ошибки:   Техническая информация для разработчика: Не удалось провести ``Заказ клиента XXXX-XXXXXX от XX.XX.XXXX XX:XX:XX``! 

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import clear_output

embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


#faiss_index_npa = FAISS.load_local("/content/drive/MyDrive/Atomic/NPA_db", embeddings, allow_dangerous_deserialization=True)
faiss_index_npa = FAISS.load_local("/content/Atomic/NPA_db", embeddings, allow_dangerous_deserialization=True)


query = "в УПП была кнопка для заполненияa колонки в ТЧ документа одним значением На скриншотах документы ``Передача материалов в кладовую``, ``Заказ материалов в производство`` возможна реализация в рамках официального ЗИ - нужно оценить трудозатраты --------------------- Добрый день! Имеется ввиду в документах по строкам «заполнение по реквизитам» , как было в УПП, например в заказе материалов не выбирать   в каждой позиции дату отгрузки.  X. В документах, например, «заказ поставщику», нет возможности как в XСERP заполнять строки аналитик «по реквизитам», просим добавить такую возможность в документы. Просьба пояснить, что имеется ввиду  -----"
query_vector = embeddings.embed_query(query)

search_results = faiss_index_npa.similarity_search(query, k=30)

In [ ]:
faiss_index_npa.similarity_search_with_score(
    "Доработка функционала возможна в рамках запроса на изменение (ЗИ). Необходимо направить новое обращение с вложением расчета экономического эффекта от реализации ЗИ."
)[:10]

[(Document(page_content='Проводки регламентной операции «Формирование финансового результата»  \nФинансовые результаты формируются регламентной операцией «Закрытие \n'),
  0.37859097),
 (Document(page_content='На основании «Заказа поставщику»  \n2. Из рабочего места «Накладные к оформлению», которое \nнаходится в Разделе «Закупки» в блоке «Закупки»  (Рисунок 167, Рисунок 168). '),
  0.46674988),
 (Document(page_content='От выбранного значения зависит \nраспределение документов факта (платежные документы  и накладные) по '),
  0.49267462),
 (Document(page_content='Во \nвкладке «Расчеты по фин. инструментам»  заполняются расчетами с арендодателями, по \nкредитам и депозитам. Если перейти в Детальные записи, то можно увидеть из каких \nдокументов состоят обороты за период. '),
  0.51159173)]

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/НПА/clean_qa.csv')

In [ ]:
df

,Unnamed: 0,Описание,Решение
0,1,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...
1,2,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...
2,3,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...
3,4,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...
4,5,XX.XX.XXXX SDXXXXXXXX требуется уточнение - во...,"Добрый день, Наталья Сергеевна Доработка функ..."
...,...,...,...
22124,22358,Добрый день. При подписании документа внутренн...,Добрый день. ч. Рекомендовано выпустить серти...
22125,22359,"Так как товар прослеживаемый, надо в карточке ...",Здравствуйте Для изменения реквизита в карточ...
22126,22360,"Коллеги, добрый день. Прошу взять в работу лис...",Добрый день. Пользователь отключен от системы ...
22127,22362,Не выводится остаточная стоимость во вкладке О...,"Здравствуйте, Владимир Олегович! Настроила для..."


In [ ]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from tqdm import tqdm

def vectorize(text):
    return embeddings.embed_query(text)

def find_top_k_nearest(index, query, k=5):
    return index.similarity_search(query, k=k)

contexts = []

for decision in tqdm(df['Решение'].fillna("")):
    if not isinstance(decision, str):
        decision = str(decision)
    decision = decision.replace("\n", " ")
    vector = vectorize(decision)
    search_results = find_top_k_nearest(faiss_index_npa, decision, k=5)
    context_texts = [result.page_content for result in search_results]
    contexts.append(context_texts)

df['Контекст'] = contexts

#df.to_excel("/content/drive/MyDrive/НПА/output_file.xlsx", index=False)
df.to_excel("/content/НПА/output_file.xlsx", index=False)

100%|██████████| 22129/22129 [05:51<00:00, 63.02it/s]


In [ ]:
#df.to_excel("/content/drive/MyDrive/НПА/output_file.xlsx", index=False)
df.to_excel("/content/НПА/output_file.xlsx", index=False)

In [ ]:
import pandas as pd
#df=pd.read_excel('/content/drive/MyDrive/НПА/output_file.xlsx')
df=pd.read_excel('/content/НПА/output_file.xlsx')

In [ ]:
df

,Unnamed: 0,Описание,Решение,Контекст
0,1,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...,['Таблица 2. Профили групп доступа \nИмя \nпр...
1,2,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...,['Проводки регламентной операции «Формирование...
2,3,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...,"['Реестр заполнится \nдокументами, которые явл..."
3,4,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...,['Проводки регламентной операции «Формирование...
4,5,XX.XX.XXXX SDXXXXXXXX требуется уточнение - во...,"Добрый день, Наталья Сергеевна Доработка функ...",['Проводки регламентной операции «Формирование...
...,...,...,...,...
22124,22358,Добрый день. При подписании документа внутренн...,Добрый день. ч. Рекомендовано выпустить серти...,['Заполнение справочника «Договоры с \nконтраг...
22125,22359,"Так как товар прослеживаемый, надо в карточке ...",Здравствуйте Для изменения реквизита в карточ...,['Полученная по результатам конкурса лицензия ...
22126,22360,"Коллеги, добрый день. Прошу взять в работу лис...",Добрый день. Пользователь отключен от системы ...,"['Если данная ситуация некорректна, то необход..."
22127,22362,Не выводится остаточная стоимость во вкладке О...,"Здравствуйте, Владимир Олегович! Настроила для...",['При формировании документа « Заявка на оплат...


In [ ]:
import re
def remove_special_characters(text):
    return re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9.,!?;:\-()«»“”’\'\"\s]', '', text)

df['Контекст'] = df['Контекст'].apply(remove_special_characters)

In [ ]:
!pip install pypdf -qU

In [ ]:
!unzip /content/model-20240615T135006Z-001.zip

Archive:  /content/model-20240615T135006Z-001.zip
   creating: model/


In [ ]:
import os
import torch
import transformers
import warnings
import pandas as pd
import json
import re
import uuid
import optuna
import requests
from concurrent.futures import ThreadPoolExecutor

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from langchain.prompts import PromptTemplate
from rouge import Rouge
from pymorphy2 import MorphAnalyzer

warnings.simplefilter('ignore')

def load_stopwords():
    try:
        with open('stopwords-ru.json', encoding='utf-8') as f:
            stop_words = json.load(f)
    except FileNotFoundError:
        response = requests.get("https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.json")
        stop_words = response.json()
        with open('stopwords-ru.json', 'w', encoding='utf-8') as f:
            json.dump(stop_words, f, ensure_ascii=False, indent=4)
    return stop_words

stop_words = load_stopwords()
morph = MorphAnalyzer()
patterns = "[«»°!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def lemmatize(string):
    clear = re.sub(patterns, ' ', string)
    tokens = [morph.normal_forms(token)[0] for token in clear.split() if token and token not in stop_words]
    return ' '.join(tokens)

def get_llm_score(answer, answer_true):
    answer = lemmatize(answer)
    answer_true = lemmatize(answer_true)
    rouge = Rouge()
    scores = rouge.get_scores(answer, answer_true)[0]
    return round(scores['rouge-1']['r'] * 100, 2)

def create_faiss_index(directory_path, embedding_model_name):
    loader = DirectoryLoader(directory_path, glob="*.pdf", loader_cls=PyPDFLoader, show_progress=True, use_multithreading=True)
    documents = loader.load()
    texts = [doc.page_content for doc in documents]
    embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)

    semantic_splitter = SemanticChunker(embedding_model)
    split_documents = semantic_splitter.create_documents(texts)
    print(f'Number of chunks created: {len(split_documents)}')
    db = FAISS.from_documents(split_documents, embedding_model)
    return db

def generate(model, tokenizer, prompt, max_new_tokens, temperature, top_p, top_k):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        bos_token_id=128000,
        eos_token_id=128001,
        pad_token_id=128001,
        do_sample=True,
        max_new_tokens=max_new_tokens,
        no_repeat_ngram_size=15,
        repetition_penalty=1.1,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

def get_llm_answer(model, tokenizer, query, chunks_join, max_new_tokens, temperature, top_p, top_k):
    user_prompt = '''Используй только следующий контекст, чтобы очень кратко ответить на вопрос в конце.
    Не пытайся выдумывать ответ.
    Контекст:
    ===========
    {chunks_join}
    ===========
    Вопрос:
    ===========
    {query}'''.format(chunks_join=chunks_join, query=query)

    SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
    RESPONSE_TEMPLATE = "assistant\n"

    prompt = f'''system\n{SYSTEM_PROMPT}\nuser\n{user_prompt}\n{RESPONSE_TEMPLATE}'''

    response = generate(model, tokenizer, prompt, max_new_tokens, temperature, top_p, top_k)

    return response
def run_one_test(df, model_name, embedding_model_name, max_new_tokens, temperature, top_p, top_k, offload_folder, db):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            offload_folder=offload_folder
        )

        retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 1})

        result = []
        for i, row in df.iterrows():
            query = row['Описание']
            answer_true = row['Решение']
            context = row['Контекст']
            response = get_llm_answer(model, tokenizer, query, context, max_new_tokens, temperature, top_p, top_k)
            row['Ответ'] = response
            row['llm_score'] = get_llm_score(response, answer_true)
            result.append(row)
            result = pd.DataFrame(result)
            result = result.sort_values(by=['llm_score'], ascending=False).reset_index(drop=True)
            score = result['llm_score'].mean()
            print(f'score{score}')
            return result, score
    except KeyError as e:
        print(f"KeyError: {e}. Проверьте, что колонки 'Описание', 'Решение' и 'Контекст' существуют в файле.")
        return None, 0
    except Exception as e:
        print(e)
        return None, 0

def objective(trial):
    global best_score, best_result

    model_name = trial.suggest_categorical('model_name', ['IlyaGusev/rugpt3medium_sum_gazeta'])
    embedding_model_name = trial.suggest_categorical('embedding_model_name', ["sentence-transformers/all-MiniLM-L6-v2"])
    max_new_tokens = trial.suggest_int('max_new_tokens', 100, 1600)
    temperature = trial.suggest_float('temperature', 0.2, 0.99)
    top_p = trial.suggest_float('top_p', 0.18, 0.99)
    top_k = trial.suggest_int('top_k', 100, 150)
    #offload_folder = '/content/drive/MyDrive/model'
    #directory_path = "/content/drive/MyDrive/НПА"
    offload_folder = '/content/model'
    directory_path = '/content/НПА'
    db = create_faiss_index(directory_path, embedding_model_name)

    result, score = run_one_test(
        TEST_DF,
        model_name,
        embedding_model_name,
        max_new_tokens, temperature, top_p, top_k, offload_folder, db
    )

    if score > best_score:
        best_score = score
        best_result = result
        best_score_tag = ' <--'
    else:
        best_score_tag = ''

    print(f'{score:.2f}', best_score_tag)
    return score

#TEST_DF=pd.read_excel('/content/drive/MyDrive/НПА/output_file.xlsx')
TEST_DF=pd.read_excel('/content/НПА/output_file.xlsx')

best_score = 0
best_result = None

study = optuna.create_study(direction='maximize')

with ThreadPoolExecutor() as executor:
    executor.submit(study.optimize, objective, n_trials=10, timeout=600)

optuna_df = study.trials_dataframe()
print('Лучшее значение:', study.best_value)
print('Среднее значение:', optuna_df['value'].mean())
print('Медианное значение:', optuna_df['value'].median())

best_params = study.best_params
best_params['df'] = TEST_DF
#directory_path = "/content/drive/MyDrive/НПА"
directory_path = '/content/НПА'


best_db = create_faiss_index(directory_path, best_params['embedding_model_name'])
result, score = run_one_test(**best_params, db=best_db)
print(result)

optuna_df.to_csv('optuna_results.csv', index=False)
best_result.to_csv('best_result.csv', index=False)

[I 2024-06-15 14:04:22,204] A new study created in memory with name: no-name-9a8e8d2e-2839-487f-9736-4b29b4fecbb9
100%|██████████| 30/30 [01:04<00:00,  2.16s/it]
[W 2024-06-15 14:05:28,967] Trial 0 failed with parameters: {'model_name': 'IlyaGusev/rugpt3medium_sum_gazeta', 'embedding_model_name': 'sentence-transformers/all-MiniLM-L6-v2', 'max_new_tokens': 835, 'temperature': 0.662918043062571, 'top_p': 0.4239052856929708, 'top_k': 107} because of the following error: RuntimeError('CUDA error: device-side assert triggered\nCUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.\nFor debugging consider passing CUDA_LAUNCH_BLOCKING=1.\nCompile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.\n').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-56-74beb8cf507f>", line 148, i

ValueError: No trials are completed yet.

NameError: name 'TORCH_USE_CUDA_DSA' is not defined